In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Vallee\\Desktop\\Stack\\myDS_projects\\tomato_disease\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Vallee\\Desktop\\Stack\\myDS_projects\\tomato_disease'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# @dataclass is a simple way to create classes that are primarily used for storing data
# frozen=True makes the dataclass immutable, meaning you cannot modify its attributes after the object is created
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from tomato_disease_classification.constants import *
from tomato_disease_classification.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [8]:
import os
import urllib.request as request
import zipfile
from tomato_disease_classification import logger
from tomato_disease_classification.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-02 16:37:40,636: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-02 16:37:40,665: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-02 16:37:40,669: INFO: common: created directory at: artifacts]
[2024-12-02 16:37:40,678: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-02 16:37:43,162: INFO: 3437825964: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 24191356
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "90634062a545e8b599f6e92e9b549dd7409f6b085864efdbeb8cf7d12c3144c9"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 585A:2FD5CA:6DBB1:7FE2C:674DE254
Accept-Ranges: bytes
Date: Mon, 02 Dec 2024 16:37:41 GMT
Via: 1.1 varnish
X-Served-By: cache-lon420147-LON
X-Cache: M